In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from matplotlib import pyplot as plt

from agents.pgp.pgp_softmax import SoftMaxPGP
from environments.gridworlds.gridworlds_classic import GridWorld
from plots.gridworlds.gridworld_visualizer import GridWorldVisualizer
from utils.policy_tools import *
from utils.policy_functions import *

In [3]:
# Agent parameters
gamma = 0.95

# World parameters
n = 4
init_locs = np.array([[1, 1]])

# Training parameters
alpha = 0.3
n_steps = 100
do_plots = False

# Actions
action_labels = np.array(["↑", "→", "↓", "←", "X"])

# Objects
world = GridWorld(n, n, init_locs)
world.A[:] = True

gpp = SoftMaxPGP(world, gamma)
viz = GridWorldVisualizer(world, gpp)

# policy softening
epsilon = 0.1

# initial position
s = gpp.state
sr = gpp.SR
ss = np.array([s])

# p(s0)
p_s = p0_onehot(gpp, s)

# action sequence
aa = np.array([1, 2, 1, 2, 0, 2])

In [4]:
# P(s2 | a) = Σ_s1 P(s2, s1 | a)
#           = Σ_s1 P(s1 | a) · P(s2 | a, s1)
#           = Σ_s1 P(s1 | a) · P(s2 | a, s1)
#           = Σ_s1 P(s1 | a) · P(a | s2, s1) · P(s2 | s1) / p(a | s1)

for i, a in enumerate(aa):

    # Do action
    s, _, _ = world.step(s, a)
    ss = np.append(ss, s)

    aa_sequence = "".join(action_labels[aa[:i+1]])
    a_current = action_labels[aa[i]]

    posterior = np.ndarray([world.n_state, world.n_state])
    p_s2 = np.zeros([world.n_state])
    
    for sP in range(world.n_state):

        # Compute Prior
        prior = np.exp(sr[sP, :]) / np.sum(np.exp(sr[sP, :]))

        # Compute likelihood
        print("STEP #{}: action sequence = {}. Considering s{} = {}".format(i+1, aa_sequence, i, world.decode(sP)))
        print("")

        likelihood = np.ndarray([world.n_state])

        for sL in range(world.n_state):

            world.reset_rewards()
            world.add_rewards( np.array([100]), np.array([sL]))
            world.A[:] = True

            gpp.reset()
            gpp.state = sP
            gpp.p0_func = (lambda agent, ss, ps, : prior/np.sum(prior))

            gpp.learn(n_steps=n_steps, alpha=alpha)
            likelihood[sL] = gpp.policy(sP)[a]
                
        # Compute posterior
        posterior[sP, :] = prior * likelihood
        posterior[sP, :] = posterior[sP, :] / np.sum(posterior[sP, :])
        p_s2 = p_s2 + posterior[sP, :] * p_s[sP]


        # Plots
        traj = np.array([sP])

        if do_plots:

            plt.subplot(231)
            viz.plot_grid(prior, plot_axis=False)
            viz.plot_trajectory(ss=traj, plot_maze=False, plot_grid=False, plot_axis=False, jitter=0)
            plt.title("p(s{} | s{} = {})".format(i+1, i, world.decode(sP)))

            plt.subplot(232)
            viz.plot_grid(likelihood, plot_axis=False)
            viz.plot_trajectory(ss=traj, plot_maze=False, plot_grid=False, plot_axis=False, jitter=0)
            plt.title("p(a = {} | s{}, s{} = {})".format(a_current, i+1, i, world.decode(sP)))


            plt.subplot(233)
            viz.plot_grid(posterior[sP, :], plot_axis=False)
            viz.plot_trajectory(ss=traj, plot_maze=False, plot_grid=False, plot_axis=False, jitter=0)
            plt.title("p(s{} | a = {}, s{} = {})".format(i+1, a_current, i, world.decode(sP)))

            plt.subplot(234)
            viz.plot_grid(p_s, plot_axis=False)
            viz.plot_trajectory(ss=traj, plot_maze=False, plot_grid=False, plot_axis=False, jitter=0)
            plt.title("p(s{} | a = {})".format(i, aa_sequence))

            plt.subplot(235)
            viz.plot_grid(p_s2, plot_axis=False)
            plt.title("partial p(s{} | a={})".format(i+1, a_current))

            plt.tight_layout()
            plt.show()
    
    p_s = p_s2


    viz.plot_grid(p_s2, plot_axis=False)
    viz.plot_trajectory(ss=ss, plot_maze=False, plot_grid=False, plot_axis=False)
    plt.title("p(s{} | a={})".format(i+1, a_current))

    plt.tight_layout()
    plt.show()
            


    





STEP #1: action sequence = →. Considering s0 = [0 0]



  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 926.91it/s]


STEP #1: action sequence = →. Considering s0 = [0 1]



100%|██████████| 100/100 [00:00<00:00, 922.85it/s]


...

STEP #1: action sequence = →. Considering s0 = [1 2]



100%|██████████| 100/100 [00:00<00:00, 1098.70it/s]


STEP #1: action sequence = →. Considering s0 = [1 3]



 11%|█         | 11/100 [00:00<00:00, 1032.92it/s]


KeyboardInterrupt: 